In [15]:
import glob

import torch
import xarray as xr
from neuralhydrology.nh_run import start_run, eval_run

from pathlib import Path

import random
random.seed(42)

### Basin file definition
select all id's where 100% of discharge observations exists

In [7]:
basins = [file.split('/')[-1][:-3] for
          file in glob.glob('../geo_data/great_db/nc_all_q/*.nc')]
with open('./openf_basins.txt', 'w') as the_file:
    for gauge_name in basins:
        the_file.write(f'{int(gauge_name)}\n')

In [10]:
random.shuffle(basins)
gauge_size = len(basins)

train_gauges = basins[:int(len(basins) * 0.8)]
with open('./basins_train.txt', 'w') as the_file:
    for gauge_name in train_gauges:
        the_file.write(f'{gauge_name}\n')

val_gauges = basins[int(gauge_size * 0.8)
                        :int(gauge_size * 0.8) + int(gauge_size * 0.1)]
with open('./basins_val.txt', 'w') as the_file:
    for gauge_name in val_gauges:
        the_file.write(f'{gauge_name}\n')

test_gauges = basins[int(gauge_size * 0.8) + int(gauge_size * 0.1):]
with open('./basins_test.txt', 'w') as the_file:
    for gauge_name in test_gauges:
        the_file.write(f'{gauge_name}\n')


### Run for different configs


#### ERA5 precipitation

In [ ]:
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/era5_prcp_qcms.yml"))

#### ERA5-Land precipitation

In [ ]:
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/era5_land_prcp_qms.yml"))

#### GPCP precipitation

In [ ]:
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/gpcp_prcp_qms.yml"))

#### IMERG precipitation

In [ ]:
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/imerg_prcp_qms.yml"))

#### MSWEP precipitation

In [ ]:
if torch.cuda.is_available():
    start_run(config_file=Path("./configs/mswep_prcp_qms.yml"))